In [12]:
import pandas as pd
import librosa
import tensorflow as tf
import numpy as np
from tqdm import tqdm


ModuleNotFoundError: No module named 'pandas'

In [ ]:

# Reading the CSV file
df = pd.read_csv('cv-valid-train.csv')
df = df[~df['gender'].isna()]

BASE_DIR = 'cv-valid-train/'
X_train, y_train = [], []

for i in tqdm(df.index):
    file = BASE_DIR + df.loc[i, 'filename']
    wf, sr = librosa.load(file)
    mfcc_wf = librosa.feature.mfcc(y=wf, sr=sr)
    b = tf.keras.utils.pad_sequences(mfcc_wf, padding='post', maxlen=200)
    X_train.append(b)
    if df.loc[i, 'gender'] == 'male':
        y_train.append(1)
    else:
        y_train.append(0)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(20, 200,)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=64, validation_split=0.2)


In [ ]:
model.save('trained_model')


In [ ]:
loaded_model = tf.keras.models.load_model('trained_model')


In [ ]:
def predict_gender(audio_file):
    wf, sr = librosa.load(audio_file)
    mfcc_wf = librosa.feature.mfcc(y=wf, sr=sr)
    b = tf.keras.utils.pad_sequences(mfcc_wf, padding='post', maxlen=200)

    prediction = loaded_model.predict(np.array([b]))

    return "Male" if prediction[0][0] > 0.5 else "Female"

In [ ]:
!pip install --upgrade gradio

In [ ]:
import gradio as gr

In [ ]:

interface = gr.Interface(
  fn=predict_gender,
  inputs=gr.Audio(),
  outputs="text",
  title="Gender Classification",
  description="Record your voice and predict your gender",
  allow_flagging=True  
)

interface.launch()


In [ ]:
b=0.7

In [ ]:
def a(b):    
    return "Male" if b > 0.5 else "Female"